In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.memory import MemorySaver

In [4]:
memory = MemorySaver()

In [5]:
from uuid import uuid4

'''
Unlike the previous implementation where we use add operator for the messages list, now we replace if the id is the same or else append in the list
'''

def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    # assign ids to messages
    for message in right:
        if not message.id:
            message.id = str(uuid4())

    # merge new message with existing messages
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            merged.append(message)
    return merged


class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

In [6]:
tool = TavilySearchResults(max_results=2)

### Manual Human Approval

In [38]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_llm)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: 'action', False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer,
                                   interrupt_before = ["action"])
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_llm(self, state:AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}
    
    def take_action(self, state:AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for tool in tool_calls:
            print(f"Calling: {tool}")
            if not tool['name'] in self.tools:
                print("\n ...bad tool name....")
                result = "bad tool name, retry"
            else:
                result = self.tools[tool['name']].invoke(tool['args'])
            results.append(ToolMessage(tool_call_id=tool['id'], name=tool['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}


In [39]:
import os

prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

# using groq api
open_ai_key = os.getenv('OPEN_API_KEY')
open_ai_base_url = "https://api.groq.com/openai/v1"
model = ChatOpenAI(model = 'llama3-8b-8192', api_key=open_ai_key, base_url=open_ai_base_url)

# initiate the agent
abot = Agent(model, [tool], checkpointer=memory, system=prompt)

In [8]:
messages = [HumanMessage(content="Whats the weather in SF?")]
thread = {"configurable": {"thread_id": "1"}}

for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='/tool-use>\n{\n  "tool_calls": [\n    {\n      "id": "pending",\n      "type": "function",\n      "function": {\n        "name": "tavily_search_results_json"\n      },\n      "parameters": {\n        "query": "weather in San Francisco"\n      }\n    }\n  ]\n}</tool-use>', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 2047, 'total_tokens': 2118, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': -0.31546127199999996, 'prompt_time': 0.253539533, 'completion_time': 0.059166667, 'total_time': 0.3127062}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'id': 'chatcmpl-679bbcb0-0c67-49d8-81b2-17890b5884bd', 'finish_reason': 'stop', 'logprobs': None}, id='run-b966fb52-6813-42e7-9e8f-d5b24f17960f-0', usage_metadata={'input_tokens': 2047, 'output_tokens': 71, 'total_tokens': 2118, 'input_token_details': {}, 'output_token_details': {}})]}


In [9]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', additional_kwargs={}, response_metadata={}, id='a00800eb-f034-44bd-a235-933bec63efcb'), AIMessage(content='/tool-use>\n{\n  "tool_calls": [\n    {\n      "id": "pending",\n      "type": "function",\n      "function": {\n        "name": "tavily_search_results_json"\n      },\n      "parameters": {\n        "query": "weather in San Francisco"\n      }\n    }\n  ]\n}</tool-use>', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 2047, 'total_tokens': 2118, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': -0.31546127199999996, 'prompt_time': 0.253539533, 'completion_time': 0.059166667, 'total_time': 0.3127062}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'id': 'chatcmpl-679bbcb0-0c67-49d8-81b2-17890b5884bd', 'finish_reason': 'stop', 'logprobs': None}, id='run-b966fb52-6813-42e7-9e8f-d5b2

In [10]:
abot.graph.get_state(thread).next

()

The agent is interupted at the action. to continue after the interrupt, you can pass the input

In [11]:
# passing input as None
for even in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

In [12]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', additional_kwargs={}, response_metadata={}, id='a00800eb-f034-44bd-a235-933bec63efcb'), AIMessage(content='/tool-use>\n{\n  "tool_calls": [\n    {\n      "id": "pending",\n      "type": "function",\n      "function": {\n        "name": "tavily_search_results_json"\n      },\n      "parameters": {\n        "query": "weather in San Francisco"\n      }\n    }\n  ]\n}</tool-use>', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 2047, 'total_tokens': 2118, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': -0.31546127199999996, 'prompt_time': 0.253539533, 'completion_time': 0.059166667, 'total_time': 0.3127062}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'id': 'chatcmpl-679bbcb0-0c67-49d8-81b2-17890b5884bd', 'finish_reason': 'stop', 'logprobs': None}, id='run-b966fb52-6813-42e7-9e8f-d5b2

In [13]:
# verifying if the exectuion is completed
abot.graph.get_state(thread).next

()

In [10]:
# automating the continue from interrupt option
messages = [HumanMessage(content="Whats the weather in CA?")]
thread = {"configurable": {"thread_id": "1"}}

for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

if abot.graph.get_state(thread).next:
    print("\n", abot.graph.get_state(thread),"\n")
    _input = input("proceed?")
    if _input != "y":
        print("aborting")
    else:
        for event in abot.graph.stream(None, thread):
            for v in event.values():
                print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5wy9', 'function': {'arguments': '{"query":"current weather in California"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 1072, 'total_tokens': 1116, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.020634650000000004, 'prompt_time': 0.13449985, 'completion_time': 0.036666667, 'total_time': 0.171166517}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'id': 'chatcmpl-c5769dda-1741-464f-a249-efa46ce14d1f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-04d34b44-dfaf-4252-8b5e-138d9b5448f7-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in California'}, 'id': 'call_5wy9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1072, 'output_tokens': 44, 'total_tokens': 1116, 'input_token_de

### Modify the state

In [41]:
messages = [HumanMessage("Whats the weather in SF?")]
thread = {"configurable": {"thread_id": "3"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_50th', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 2136, 'total_tokens': 2178, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.02402025400000002, 'prompt_time': 0.264434214, 'completion_time': 0.035, 'total_time': 0.299434214}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'id': 'chatcmpl-518a479c-6676-44d5-b5d0-29ca7664f577', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f0d42931-ab8f-4d1e-8cff-7c8692f2d6df-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_50th', 'type': 'tool_call'}], usage_metadata={'input_tokens': 2136, 'output_tokens': 42, 'total_tokens': 2178, 'input_token_details': {}, 'out

In [42]:
# get_state is used for getting the current state
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='9fc145e9-c04c-4722-8b2e-0c11addd826c'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rwsg', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 2047, 'total_tokens': 2120, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': -0.310704767, 'prompt_time': 0.254771177, 'completion_time': 0.060833333, 'total_time': 0.31560451}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'id': 'chatcmpl-eaa4b77a-872f-4e3d-918b-0c8ef6db003a', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ec1c2552-1f0a-4a18-af7d-c998fe412eea-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'i

In [43]:
current_values = abot.graph.get_state(thread)

In [44]:
current_values.values['messages'][-1]

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_50th', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 2136, 'total_tokens': 2178, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.02402025400000002, 'prompt_time': 0.264434214, 'completion_time': 0.035, 'total_time': 0.299434214}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'id': 'chatcmpl-518a479c-6676-44d5-b5d0-29ca7664f577', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f0d42931-ab8f-4d1e-8cff-7c8692f2d6df-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_50th', 'type': 'tool_call'}], usage_metadata={'input_tokens': 2136, 'output_tokens': 42, 'total_tokens': 2178, 'input_token_details': {}, 'output_token_deta

In [45]:
current_values.values['messages'][-1].tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'weather in San Francisco'},
  'id': 'call_50th',
  'type': 'tool_call'}]

In [46]:
# modify the current state
_id = current_values.values['messages'][-1].tool_calls[0]['id']
current_values.values['messages'][-1].tool_calls = [{'name': 'tavily_search_results_json',
  'args': {'query': 'weather in New delhi'},
  'id': _id,
  'type': 'tool_call'}]

In [47]:
# update the agent with the new state
abot.graph.update_state(thread, current_values.values)

{'configurable': {'thread_id': '3',
  'checkpoint_ns': '',
  'checkpoint_id': '1f007944-20cd-63e4-8013-a8488cfa6123'}}

In [48]:
abot.graph.get_state(thread).values['messages'][-1].tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'weather in New delhi'},
  'id': 'call_50th',
  'type': 'tool_call'}]

In [49]:
for event in abot.graph.stream(None, thread):
    for k, v in event.items():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in New delhi'}, 'id': 'call_50th', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'title\': \'Weather in New Delhi, India\', \'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'New Delhi\', \'region\': \'Delhi\', \'country\': \'India\', \'lat\': 28.6, \'lon\': 77.2, \'tz_id\': \'Asia/Kolkata\', \'localtime_epoch\': 1742699383, \'localtime\': \'2025-03-23 08:39\'}, \'current\': {\'last_updated_epoch\': 1742698800, \'last_updated\': \'2025-03-23 08:30\', \'temp_c\': 22.3, \'temp_f\': 72.1, \'is_day\': 1, \'condition\': {\'text\': \'Mist\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/143.png\', \'code\': 1030}, \'wind_mph\': 3.1, \'wind_kph\': 5.0, \'wind_degree\': 137, \'wind_dir\': \'SE\', \'pressure_mb\': 1016.0, \'pressure_in\': 30.0, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 50, \'cloud\': 0, \'feelslike_c\': 22.2, \'feelslike_f\

### Time travel

In [50]:
states = []
# get_state_history is used to get all the prev histories
for state in abot.graph.get_state_history(thread):
    print(state)
    print('--')
    states.append(state)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='9fc145e9-c04c-4722-8b2e-0c11addd826c'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rwsg', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 2047, 'total_tokens': 2120, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': -0.310704767, 'prompt_time': 0.254771177, 'completion_time': 0.060833333, 'total_time': 0.31560451}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'id': 'chatcmpl-eaa4b77a-872f-4e3d-918b-0c8ef6db003a', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ec1c2552-1f0a-4a18-af7d-c998fe412eea-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'i

In [51]:
to_reply = states[-3]

In [52]:
to_reply

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='9fc145e9-c04c-4722-8b2e-0c11addd826c'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rwsg', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 2047, 'total_tokens': 2120, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': -0.310704767, 'prompt_time': 0.254771177, 'completion_time': 0.060833333, 'total_time': 0.31560451}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'id': 'chatcmpl-eaa4b77a-872f-4e3d-918b-0c8ef6db003a', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ec1c2552-1f0a-4a18-af7d-c998fe412eea-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'i

In [53]:
for event in abot.graph.stream(None, to_reply.config):
    for k, v in event.items():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_rwsg', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content="[{'title': 'Los Angeles weather in March 2025 - California - Weather2Travel.com', 'url': 'https://www.weather2travel.com/california/los-angeles/march/', 'content': 'Los Angeles weather in March 2025 | California, USA\\n\\n\\nWeather\\nDeals\\nInspiration\\nNews\\nTravel advice\\nHolidays\\nExtras\\nInsights\\n\\n\\n\\n\\nHome\\nNorth America\\nUSA\\nCalifornia\\nLos Angeles\\nWhere is hot in March?\\n\\nLos Angeles weather in March 2025\\nExpect 21°C daytime maximum temperatures in the shade with on average 9 hours of sunshine per day in Los Angeles in March. Check more long-term weather averages for Los Angeles in March before you book your next holiday to California in 2025/2026.', 'score': 0.87848678125}, {'title': 'Weather in Los Angeles in March 2025 - Detailed Forecast', 'url': 'https://www.easew

### Go back in time and edit

In [54]:
to_reply

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='9fc145e9-c04c-4722-8b2e-0c11addd826c'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rwsg', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 2047, 'total_tokens': 2120, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': -0.310704767, 'prompt_time': 0.254771177, 'completion_time': 0.060833333, 'total_time': 0.31560451}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'id': 'chatcmpl-eaa4b77a-872f-4e3d-918b-0c8ef6db003a', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ec1c2552-1f0a-4a18-af7d-c998fe412eea-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'i

In [55]:
_id = to_reply.values['messages'][-1].tool_calls[0]['id']
to_reply.values['messages'][-1].tool_calls = [{'name': 'tavily_search_results_json',
  'args': {'query': 'current weather in LA, accuweather'},
  'id': _id}]

In [56]:
branch_state = abot.graph.update_state(to_reply.config, to_reply.values)

In [57]:
for event in abot.graph.stream(None, branch_state):
    for k, v in event.items():
        if k != "__end__":
            print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in LA, accuweather'}, 'id': 'call_rwsg', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'title\': \'Weather in los angeles ca\', \'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Los Angeles\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 34.0522, \'lon\': -118.2428, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1742699408, \'localtime\': \'2025-03-22 20:10\'}, \'current\': {\'last_updated_epoch\': 1742698800, \'last_updated\': \'2025-03-22 20:00\', \'temp_c\': 17.2, \'temp_f\': 63.0, \'is_day\': 0, \'condition\': {\'text\': \'Clear\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/113.png\', \'code\': 1000}, \'wind_mph\': 3.4, \'wind_kph\': 5.4, \'wind_degree\': 206, \'wind_dir\': \'SSW\', \'pressure_mb\': 1014.0, \'pressure_in\': 29.93, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\

### Add a message to state at a given time

In [58]:
to_reply

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', additional_kwargs={}, response_metadata={}, id='9fc145e9-c04c-4722-8b2e-0c11addd826c'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rwsg', 'function': {'arguments': '{"query":"weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 2047, 'total_tokens': 2120, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': -0.310704767, 'prompt_time': 0.254771177, 'completion_time': 0.060833333, 'total_time': 0.31560451}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'id': 'chatcmpl-eaa4b77a-872f-4e3d-918b-0c8ef6db003a', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ec1c2552-1f0a-4a18-af7d-c998fe412eea-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in LA, accuw

In [60]:
_id = to_reply.values['messages'][-1].tool_calls[0]['id']

In [61]:
state_update = {"messages": [ToolMessage(
    tool_call_id=_id,
    name="tavily_search_results_json",
    content="100 degree celcius",
)]}

In [62]:
# we are updating the state along with as_node = "action", to mention this as the the response from the action
branch_and_add = abot.graph.update_state(
    to_reply.config, 
    state_update, 
    as_node="action")

In [63]:
for event in abot.graph.stream(None, branch_and_add):
    for k, v in event.items():
        print(v)

{'messages': [AIMessage(content="It seems like it's extremely hot in LA right now, with a temperature of 100 degrees Celsius!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 2192, 'total_tokens': 2214, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': -0.396260905, 'prompt_time': 0.368945795, 'completion_time': 0.018333333, 'total_time': 0.387279128}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'id': 'chatcmpl-fb423d80-bf87-48ef-912b-9ed627839754', 'finish_reason': 'stop', 'logprobs': None}, id='run-d3b0faf8-b22c-410d-9c85-e6c2b284e3f0-0', usage_metadata={'input_tokens': 2192, 'output_tokens': 22, 'total_tokens': 2214, 'input_token_details': {}, 'output_token_details': {}})]}


Thus you can see the change in the temperature